# TISCA Example Usage with User-Defined Simulation

This script demonstrates how to use the TISCA framework with a custom simulation function. It provides a complete working example that researchers can adapt for their own studies

In [1]:
source("TISCA.R")

# Load required packages for this example
library(MASS)  # For mvrnorm function

Example: Simple Treatment Effect Estimation Simulation

This example simulates a basic treatment effect estimation scenario with:
 - Two competing methods: a proposed method and a benchmark
 - Two performance metrics: PEHE (lower is better) and Coverage (higher is better)
 - Simple data generation process with heterogeneous treatment effects


In [2]:
#' @param seed Random seed for reproducibility
#' @return A data frame with performance metrics for both methods
simulate_treatment_effects <- function(seed) {
  # Set random seed for reproducibility
  set.seed(seed)
  
  # 1. Generate synthetic data
  n <- 100  # Sample size
  p <- 5    # Number of covariates
  
  # Generate covariates
  X <- matrix(rnorm(n * p), nrow = n, ncol = p)
  
  # Generate true treatment effects (heterogeneous)
  tau <- 2 * X[,1] + X[,2]^2
  
  # Generate treatment assignment
  propensity <- plogis(0.5 + 0.3 * X[,1] - 0.2 * X[,3])
  Z <- rbinom(n, 1, propensity)
  
  # Generate outcomes
  mu <- 1 + X %*% c(1, 0.5, -0.5, 1, 2)  # Baseline effect
  Y <- mu + Z * tau + rnorm(n, 0, 1)     # Observed outcome
  
  # 2. Split data into training and testing
  train_idx <- sample(1:n, 0.8 * n)
  test_idx <- setdiff(1:n, train_idx)
  
  X_train <- X[train_idx, ]
  Z_train <- Z[train_idx]
  Y_train <- Y[train_idx]
  
  X_test <- X[test_idx, ]
  Z_test <- Z[test_idx]
  Y_test <- Y[test_idx]
  tau_test <- tau[test_idx]  # True treatment effects for evaluation
  
  # 3. Fit models
  
  # Proposed method: More complex model (simulated)
  # In a real application, this would be your novel method
  proposed_fit <- function() {
    # Simulate a more accurate but variable model
    tau_hat <- tau_test + rnorm(length(test_idx), 0, 0.8)
    
    # Generate posterior samples for uncertainty quantification
    posterior_samples <- matrix(NA, nrow = 100, ncol = length(test_idx))
    for (i in 1:100) {
      posterior_samples[i, ] <- tau_hat + rnorm(length(test_idx), 0, 0.7)
    }
    
    return(list(
      point_estimates = tau_hat,
      posterior_samples = posterior_samples
    ))
  }
  
  # Benchmark method: Simpler model (simulated)
  # In a real application, this would be a standard method
  benchmark_fit <- function() {
    # Simulate a less accurate but more stable model
    tau_hat <- tau_test + rnorm(length(test_idx), 0.2, 1.0)
    
    # Generate posterior samples for uncertainty quantification
    posterior_samples <- matrix(NA, nrow = 100, ncol = length(test_idx))
    for (i in 1:100) {
      posterior_samples[i, ] <- tau_hat + rnorm(length(test_idx), 0, 0.9)
    }
    
    return(list(
      point_estimates = tau_hat,
      posterior_samples = posterior_samples
    ))
  }
  
  # 4. Get model predictions
  proposed_results <- proposed_fit()
  benchmark_results <- benchmark_fit()
  
  # 5. Calculate performance metrics
  
  # PEHE (Precision in Estimation of Heterogeneous Effects)
  proposed_pehe <- sqrt(mean((proposed_results$point_estimates - tau_test)^2))
  benchmark_pehe <- sqrt(mean((benchmark_results$point_estimates - tau_test)^2))
  
  # Coverage of 95% credible intervals
  calculate_coverage <- function(posterior_samples, true_values) {
    lower <- apply(posterior_samples, 2, function(x) quantile(x, 0.025))
    upper <- apply(posterior_samples, 2, function(x) quantile(x, 0.975))
    mean(true_values >= lower & true_values <= upper)
  }
  
  proposed_coverage <- calculate_coverage(proposed_results$posterior_samples, tau_test)
  benchmark_coverage <- calculate_coverage(benchmark_results$posterior_samples, tau_test)
  
  # 6. Return results as a data frame
  return(data.frame(
    proposed_pehe = proposed_pehe,
    benchmark_pehe = benchmark_pehe,
    proposed_coverage = proposed_coverage,
    benchmark_coverage = benchmark_coverage
  ))
}

In [3]:
# Test the simulation function with a single run
test_run <- simulate_treatment_effects(seed = 42)
print("Single simulation run results:")
print(test_run)

[1] "Single simulation run results:"
  proposed_pehe benchmark_pehe proposed_coverage benchmark_coverage
1     0.6960714      0.7112506              0.95               0.95


In [4]:
# Define comparison pairs for TISCA
# Format: list of c("proposed_metric", "benchmark_metric") pairs
comparison_pairs <- list(
  c("proposed_pehe", "benchmark_pehe"),         # Compare PEHE (lower is better)
  c("proposed_coverage", "benchmark_coverage")  # Compare coverage (higher is better)
)

In [5]:
# Define minimum detectable effect sizes (MDEs)
# For PEHE: -0.1 means proposed is at least 0.1 better
# For coverage: 0.05 means proposed is at least 0.05 closer to 0.95
mdes <- c(-0.1, 0.05)

In [6]:
# Run TISCA with the simulation function
cat("\nRunning TISCA to determine optimal simulation count...\n\n")

tisca_results <- run_tisca(
  sim_func = simulate_treatment_effects,
  comparison_pairs = comparison_pairs,
  mdes = mdes,
  target_power = 0.90,     # Target statistical power
  alpha = 0.01,            # Significance level
  batch_size = 10,         # Number of simulations per batch (small for demonstration)
  initial_count = 10,      # Initial simulations before first power check
  correction_method = "holm",  # Multiple testing correction method
  verbose = TRUE,          # Print progress updates
  save_results = TRUE      # Save results to CSV files
)

# Print summary table
cat("\nTISCA Summary Table:\n")
print(tisca_results$summary_table)


Running TISCA to determine optimal simulation count...

Starting TISCA algorithm...
Target power: 0.9 
Significance level: 0.01 
Batch size: 10 
Initial count: 10 
Correction method: holm 
Number of comparisons: 2 

Running initial 10 simulations...

After initial 10 simulations:
Comparison 1: Power = 0.1537, p-value = 0.0002, adjusted p-value = 0.0005
Comparison 2: Power = 0.1988, p-value = 0.1043, adjusted p-value = 0.1043
Minimum power: 0.1537198 

Iteration 1: Running batch of 10 simulations (total will be 20)...

After 20 simulations:
Comparison 1: Power = 0.3323, p-value = 0.0000, adjusted p-value = 0.0000
Comparison 2: Power = 0.5889, p-value = 0.0121, adjusted p-value = 0.0121
Minimum power: 0.3323259 

Iteration 2: Running batch of 10 simulations (total will be 30)...

After 30 simulations:
Comparison 1: Power = 0.4812, p-value = 0.0000, adjusted p-value = 0.0000
Comparison 2: Power = 0.7292, p-value = 0.0253, adjusted p-value = 0.0253
Minimum power: 0.481195 

Iteration 3: R

Warning message in pt(crit, df, ncp):
“precisão completa pode não ser conseguida em 'pnt{final}'”



After 60 simulations:
Comparison 1: Power = 0.9030, p-value = 0.0000, adjusted p-value = 0.0000
Comparison 2: Power = 0.9743, p-value = 0.1712, adjusted p-value = 0.1712
Minimum power: 0.9029805 


=== TISCA COMPLETED ===
Final number of simulations required: 60 
Final achieved powers:
Comparison 1 (proposed_pehe vs benchmark_pehe): Power = 0.9030, p-value = 0.0000, adjusted p-value = 0.0000
Comparison 2 (proposed_coverage vs benchmark_coverage): Power = 0.9743, p-value = 0.1712, adjusted p-value = 0.1712

Results saved to CSV files:
-  ./tisca_simulation_results.csv 
-  ./tisca_summary_results.csv 
-  ./power_tracking.csv 
-  ./pvalue_tracking.csv 

Visualizations saved:
-  ./power_vs_simulations.png 
-  ./pvalue_comparison.png 

TISCA Summary Table:
                               Comparison   Proposed_Metric   Benchmark_Metric
1         proposed_pehe vs benchmark_pehe     proposed_pehe     benchmark_pehe
2 proposed_coverage vs benchmark_coverage proposed_coverage benchmark_coverage


In [7]:
# Create visualizations of the results
if (requireNamespace("ggplot2", quietly = TRUE)) {
  library(ggplot2)
  library(reshape2)
  
  # 1. Plot PEHE values across simulations
  pehe_data <- melt(tisca_results$results[, c("proposed_pehe", "benchmark_pehe")], 
                    variable.name = "Method", value.name = "PEHE")
  
  pehe_plot <- ggplot(pehe_data, aes(x = Method, y = PEHE, fill = Method)) +
    geom_boxplot() +
    theme_classic() +
    labs(title = "Comparison of PEHE Values",
         subtitle = paste("Based on", tisca_results$J_final, "simulations"),
         y = "PEHE (lower is better)") +
    scale_fill_brewer(palette = "Set1")
  
  # 2. Plot coverage values across simulations
  coverage_data <- melt(tisca_results$results[, c("proposed_coverage", "benchmark_coverage")], 
                        variable.name = "Method", value.name = "Coverage")
  
  coverage_plot <- ggplot(coverage_data, aes(x = Method, y = Coverage, fill = Method)) +
    geom_boxplot() +
    theme_classic() +
    labs(title = "Comparison of Coverage Values",
         subtitle = paste("Based on", tisca_results$J_final, "simulations"),
         y = "Coverage (higher is better)") +
    scale_fill_brewer(palette = "Set1") +
    coord_cartesian(ylim = c(0.7, 1.0))
  
  # Save plots
  ggsave("pehe_comparison.png", pehe_plot, width = 8, height = 6)
  ggsave("coverage_comparison.png", coverage_plot, width = 8, height = 6)
  
  cat("\nVisualization plots saved as 'pehe_comparison.png' and 'coverage_comparison.png'\n")
}

No id variables; using all as measure variables

No id variables; using all as measure variables




Visualization plots saved as 'pehe_comparison.png' and 'coverage_comparison.png'


In [8]:
# Demonstrate how to access and use TISCA results
cat("\nAccessing TISCA results programmatically:\n")
cat("Required number of simulations:", tisca_results$J_final, "\n")
cat("Achieved power for PEHE comparison:", tisca_results$P_achieved[1], "\n")
cat("Achieved power for coverage comparison:", tisca_results$P_achieved[2], "\n")
cat("Raw p-value for PEHE comparison:", tisca_results$P_raw[1], "\n")
cat("Adjusted p-value for PEHE comparison:", tisca_results$P_adj[1], "\n")


Accessing TISCA results programmatically:
Required number of simulations: 60 
Achieved power for PEHE comparison: 0.9029805 
Achieved power for coverage comparison: 0.9743016 
Raw p-value for PEHE comparison: 3.774627e-15 
Adjusted p-value for PEHE comparison: 7.549254e-15 


In [9]:
# Example of how to report results in a publication
cat("\nExample text for reporting in a publication:\n\n")
cat(paste0(
  "We conducted a simulation study to compare our proposed method with the benchmark approach. ",
  "Using the Test-Informed Simulation Count Algorithm (TISCA), we determined that ", 
  tisca_results$J_final, " simulations were required to achieve 80% power for detecting ",
  "a minimum difference of 0.1 in PEHE and 0.05 in coverage at a significance level of 0.05. ",
  "Our proposed method achieved significantly ", 
  ifelse(tisca_results$P_adj[1] < 0.05, "lower", "similar"), " PEHE ",
  "(", round(mean(tisca_results$results$proposed_pehe), 3), " vs. ", 
  round(mean(tisca_results$results$benchmark_pehe), 3), ", adjusted p-value = ", 
  round(tisca_results$P_adj[1], 4), ") and significantly ",
  ifelse(tisca_results$P_adj[2] < 0.05, "higher", "similar"), " coverage ",
  "(", round(mean(tisca_results$results$proposed_coverage), 3), " vs. ", 
  round(mean(tisca_results$results$benchmark_coverage), 3), ", adjusted p-value = ", 
  round(tisca_results$P_adj[2], 4), ")."
))



Example text for reporting in a publication:

We conducted a simulation study to compare our proposed method with the benchmark approach. Using the Test-Informed Simulation Count Algorithm (TISCA), we determined that 60 simulations were required to achieve 80% power for detecting a minimum difference of 0.1 in PEHE and 0.05 in coverage at a significance level of 0.05. Our proposed method achieved significantly lower PEHE (0.781 vs. 1.011, adjusted p-value = 0) and significantly similar coverage (0.902 vs. 0.887, adjusted p-value = 0.1712).